In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

import xgboost as xgb

# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

Getting Files open and into dataframes

In [3]:
file_path = "playground-series-s4e11//"

df_train = pd.read_csv(file_path + "train.csv")
df_test = pd.read_csv(file_path + "test.csv" )

Truncating df_train

In [4]:
#DF Train

df_train['Occupation Satisfaction'] = df_train['Study Satisfaction'].combine_first(df_train['Job Satisfaction'])
df_train['Overall Pressure'] = df_train['Academic Pressure'].combine_first(df_train['Work Pressure'])
df_train = df_train.drop(['Study Satisfaction','Job Satisfaction','Academic Pressure', 'Work Pressure','CGPA'], axis=1)


df_train_modified = df_train.drop(['Name', 'Depression','id','City'], axis=1)
df_dummy_train = pd.get_dummies(df_train_modified,drop_first=True,dtype=float)


y = df_train['Depression']

In [5]:
df_dummy_train

,Age,Work/Study Hours,Financial Stress,Occupation Satisfaction,Overall Pressure,Gender_Male,Working Professional or Student_Working Professional,Profession_Accountant,Profession_Analyst,Profession_Architect,...,Degree_S.Pharm,Degree_S.Tech,Degree_UX/UI Designer,Degree_Unite,Degree_Veda,Degree_Vivaan,Degree_Vrinda,Degree_Working Professional,Have you ever had suicidal thoughts ?_Yes,Family History of Mental Illness_Yes
0,49.0,1.0,2.0,2.0,5.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,26.0,7.0,3.0,3.0,4.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,33.0,3.0,1.0,2.0,5.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,22.0,10.0,1.0,1.0,5.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,30.0,9.0,4.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140695,18.0,2.0,4.0,4.0,5.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
140696,41.0,6.0,5.0,4.0,5.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
140697,24.0,4.0,4.0,1.0,3.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140698,49.0,10.0,1.0,2.0,5.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Truncating df_test

In [6]:
df_test['Occupation Satisfaction'] = df_test['Study Satisfaction'].combine_first(df_test['Job Satisfaction'])
df_test['Overall Pressure'] = df_test['Academic Pressure'].combine_first(df_test['Work Pressure'])
df_test = df_test.drop(['Study Satisfaction','Job Satisfaction','Academic Pressure', 'Work Pressure','CGPA'], axis=1)


df_test_modified = df_test.drop(['Name','id','City'], axis=1)
df_dummy_test = pd.get_dummies(df_test_modified,drop_first=True,dtype=float)
df_dummy_test



,Age,Work/Study Hours,Financial Stress,Occupation Satisfaction,Overall Pressure,Gender_Male,Working Professional or Student_Working Professional,Profession_3M,Profession_Accountant,Profession_Analyst,...,Degree_PhD,Degree_Pune,Degree_RCA,Degree_Rupak,Degree_S.Pharm,Degree_Travel Consultant,Degree_Vibha,Degree_Vrinda,Have you ever had suicidal thoughts ?_Yes,Family History of Mental Illness_Yes
0,53.0,9.0,3.0,5.0,2.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,58.0,6.0,4.0,4.0,2.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,53.0,12.0,4.0,1.0,4.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,23.0,10.0,4.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,47.0,3.0,4.0,5.0,5.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93795,49.0,2.0,2.0,5.0,3.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
93796,29.0,11.0,3.0,1.0,5.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
93797,24.0,7.0,1.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93798,23.0,7.0,5.0,2.0,4.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [7]:
# Get all columns present in either df_train or df_test
def align_dataframes(df1, df2, fill_value=0):
    # Get the union of all columns
    all_columns = set(df1.columns).union(set(df2.columns))
    
    # Add missing columns and fill with `fill_value`
    for col in all_columns:
        if col not in df1.columns:
            df1[col] = fill_value
        if col not in df2.columns:
            df2[col] = fill_value
            
    # Reorder columns to ensure consistency
    df1 = df1[sorted(all_columns)]
    df2 = df2[sorted(all_columns)]
    
    return df1, df2

df_dummy_train, df_dummy_test = align_dataframes(df_dummy_train, df_dummy_test)

df_dummy_train

/var/folders/xs/7cmd8k6907zbj8nkv6m694rc0000gn/T/ipykernel_69502/878029721.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2[col] = fill_value
/var/folders/xs/7cmd8k6907zbj8nkv6m694rc0000gn/T/ipykernel_69502/878029721.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2[col] = fill_value
/var/folders/xs/7cmd8k6907zbj8nkv6m694rc0000gn/T/ipykernel_69502/878029721.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consi

,Age,Degree_20,Degree_24,Degree_29,Degree_3.0,Degree_5.56,Degree_5.61,Degree_5.65,Degree_5.88,Degree_7.06,...,Sleep Duration_More than 8 hours,Sleep Duration_No,Sleep Duration_Pune,Sleep Duration_Sleep_Duration,Sleep Duration_Unhealthy,Sleep Duration_Vivan,Sleep Duration_Work_Study_Hours,Sleep Duration_than 5 hours,Work/Study Hours,Working Professional or Student_Working Professional
0,49.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,1.0
1,26.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,7.0,1.0
2,33.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,3.0,0.0
3,22.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,10.0,1.0
4,30.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,9.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140695,18.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,2.0,1.0
140696,41.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,6.0,1.0
140697,24.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0,0.0,0.0,4.0,1.0
140698,49.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,10.0,1.0


In [8]:
x_train, x_test, y_train, y_test = train_test_split(df_dummy_train, y, stratify=y, random_state=23)

In [9]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( x_train, y_train), ( x_test, y_test)]
    
    clf.fit(x_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(x_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 10,
                        trials = trials)
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/opt/homebrew/lib/python3.12/site-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

/opt/homebrew/lib/python3.12/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                
0.8182800284292822                                    
 10%|█         | 1/10 [00:01<00:17,  1.94s/trial, best loss: -0.8182800284292822]

/opt/homebrew/lib/python3.12/site-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

/opt/homebrew/lib/python3.12/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                           
0.8182800284292822                                                               
 20%|██        | 2/10 [00:04<00:17,  2.20s/trial, best loss: -0.8182800284292822]

...

SCORE:                                                                           
0.8182800284292822                                                               
 80%|████████  | 8/10 [00:15<00:03,  1.78s/trial, best loss: -0.8182800284292822]

/opt/homebrew/lib/python3.12/site-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

/opt/homebrew/lib/python3.12/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                           
0.8182800284292822                                                               
 90%|█████████ | 9/10 [00:17<00:01,  1.89s/trial, best loss: -0.8182800284292822]

/opt/homebrew/lib/python3.12/site-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

/opt/homebrew/lib/python3.12/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                           
0.8182800284292822                                                               
100%|██████████| 10/10 [00:21<00:00,  2.11s/trial, best loss: -0.8182800284292822]
The best hyperparameters are :  

{'colsample_bytree': 0.5934356619005228, 'gamma': 4.190479544038061, 'max_depth': 11.0, 'min_child_weight': 5.0, 'reg_alpha': 76.0, 'reg_lambda': 0.015746046892539667}


In [23]:
model = xgb.XGBClassifier(learning_rate =0.1,
    n_estimators=1000,
    early_stopping_rounds=50,
    max_depth=11,
    min_child_weight=5,
    gamma=4.19,
    colsample_bytree=0.59,
    subsample=0.65,
    reg_alpha=76,
    reg_lambda = 0.015,
    objective= 'binary:hinge',
    nthread=-1,
    random_state=42)


In [24]:
model.fit(x_train, y_train, verbose=True, eval_set=[(x_test, y_test)])
y_pred = model.predict(x_test)

[0]	validation_0-error:0.81828
[1]	validation_0-error:0.81828
[2]	validation_0-error:0.81828
[3]	validation_0-error:0.81828
[4]	validation_0-error:0.81828
[5]	validation_0-error:0.27841
[6]	validation_0-error:0.17603
[7]	validation_0-error:0.12239
[8]	validation_0-error:0.11096
[9]	validation_0-error:0.10050
[10]	validation_0-error:0.09328
[11]	validation_0-error:0.08426
[12]	validation_0-error:0.07886
[13]	validation_0-error:0.07710
[14]	validation_0-error:0.07696
[15]	validation_0-error:0.07707
[16]	validation_0-error:0.07537
[17]	validation_0-error:0.07412
[18]	validation_0-error:0.07198
[19]	validation_0-error:0.07104
[20]	validation_0-error:0.06996
[21]	validation_0-error:0.06985
[22]	validation_0-error:0.06942
[23]	validation_0-error:0.06894
[24]	validation_0-error:0.06837
[25]	validation_0-error:0.06800
[26]	validation_0-error:0.06826
[27]	validation_0-error:0.06778
[28]	validation_0-error:0.06724
[29]	validation_0-error:0.06701
[30]	validation_0-error:0.06692
[31]	validation_0-

In [15]:
#y_pred_labels = np.argmax(y_pred, axis=1)
print(f'y_test shape: {y_test.shape}')
print(f'y_pred shape: {y_pred.shape}')

y_test shape: (35175,)
y_pred shape: (35175,)


In [25]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

print('\nClassification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.94

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.97      0.96     28783
           1       0.84      0.80      0.82      6392

    accuracy                           0.94     35175
   macro avg       0.90      0.88      0.89     35175
weighted avg       0.94      0.94      0.94     35175



In [26]:
importances = model.feature_importances_
columns = df_dummy_train.columns
i = 0

while i< len(columns):
    if importances[i]*100 > 1.0:
        print (f" The importance of feature '{columns[i]}' is {round(importances[i]*100, 2)}%.")
    i += 1

 The importance of feature 'Age' is 13.8%.
 The importance of feature 'Degree_Class 12' is 17.71%.
 The importance of feature 'Dietary Habits_Unhealthy' is 1.1%.
 The importance of feature 'Financial Stress' is 2.64%.
 The importance of feature 'Have you ever had suicidal thoughts ?_Yes' is 26.86%.
 The importance of feature 'Occupation Satisfaction' is 1.52%.
 The importance of feature 'Overall Pressure' is 5.72%.
 The importance of feature 'Profession_Consultant' is 1.02%.
 The importance of feature 'Profession_Teacher' is 2.63%.
 The importance of feature 'Work/Study Hours' is 1.36%.
 The importance of feature 'Working Professional or Student_Working Professional' is 15.24%.


Predicting x_test now

In [27]:
predictions = model.predict(df_dummy_test)
predictions

array([0, 0, 0, ..., 0, 1, 0])

In [28]:
df_submission = pd.DataFrame({
    'id':df_test['id'],
    'Depression': predictions
})

In [29]:
df_submission.to_csv("Submission#3_XGboost_Hinge.csv", index=False)